In [1]:
%env CUDA_VISIBLE_DEVICES 1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import torch
import torchvision
from torchvision import transforms as T
import numpy as np
from tqdm import tqdm_notebook
from PIL import Image
from glob import glob
import shutil
from sklearn.cluster import KMeans

In [3]:
from model import build_model
from dataset import make_dataloader
from config import cfg

In [4]:
cfg.merge_from_file('configs/cvwc1.yml')

In [5]:
_, _, _, num_classes = make_dataloader(cfg)

=> Data loaded
Dataset statistics:
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   107 |     1887 |         1
  query    |    20 |       40 |         1
  gallery  |    20 |      438 |         1
  ----------------------------------------


In [6]:
model = build_model(cfg, num_classes)
para_dict = torch.load('outputs/cvwc_local/resnet50_epoch45.pth')
model.load_state_dict(para_dict)
model.cuda()
model.eval();

In [7]:
trm = T.Compose([
    T.Resize(cfg.INPUT.SIZE_TEST),
    T.ToTensor(),
    T.Normalize(mean=cfg.INPUT.PIXEL_MEAN, std=cfg.INPUT.PIXEL_STD),
])

In [8]:
gallery_list = sorted(glob('/home/zbc/data/cvwc/reid_test/test/*.jpg'))
query_list = sorted(glob('/home/zbc/data/cvwc/reid_test/test/*.jpg'))
img_list = query_list
gallery_np = np.array(gallery_list)
query_np = np.array(query_list)
len(query_list), len(gallery_list)

(1764, 1764)

In [9]:
feats = []
for im in tqdm_notebook(img_list):
    im = Image.open(im)
    feat = model(trm(im).unsqueeze(0).cuda())
    feats.append(feat.cpu().data)
del feat
feats = torch.cat(feats)

In [10]:
query_feat = feats
gallery_feat = feats

In [11]:
query_feat.shape, gallery_feat.shape

(torch.Size([1764, 2048]), torch.Size([1764, 2048]))

In [12]:
from evaluate import euclidean_dist
from evaluate import re_rank

In [13]:
#distmat = euclidean_dist(query_feat, gallery_feat)
distmat = re_rank(query_feat, gallery_feat)
ind = np.argsort(distmat, axis=1)

In [24]:
distmat[0][ind[0]][:10]

array([4.1723251e-08, 2.4751529e-01, 2.5279206e-01, 2.6545784e-01,
       2.7384782e-01, 2.7899763e-01, 2.9523459e-01, 3.1298691e-01,
       3.1582478e-01, 3.6392900e-01], dtype=float32)

In [25]:
gallery_np[ind[0]][:10]

array(['/home/zbc/data/cvwc/reid_test/test/000000.jpg',
       '/home/zbc/data/cvwc/reid_test/test/003349.jpg',
       '/home/zbc/data/cvwc/reid_test/test/001191.jpg',
       '/home/zbc/data/cvwc/reid_test/test/005074.jpg',
       '/home/zbc/data/cvwc/reid_test/test/000483.jpg',
       '/home/zbc/data/cvwc/reid_test/test/003810.jpg',
       '/home/zbc/data/cvwc/reid_test/test/004558.jpg',
       '/home/zbc/data/cvwc/reid_test/test/000067.jpg',
       '/home/zbc/data/cvwc/reid_test/test/003746.jpg',
       '/home/zbc/data/cvwc/reid_test/test/002555.jpg'], dtype='<U45')

In [26]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [27]:
from utils.vistools import read_im, make_im_grid, save_im

In [28]:
def show(q_id=0, show_num=10):
    q_im = read_im(query_np[q_id])
    g_ims = [read_im(i) for i in gallery_np[ind[q_id]][1:show_num+1]]
    ims = [q_im]
    ims.extend(g_ims)
    import math
    s = int(math.sqrt(len(ims))) + 1
    img = make_im_grid(ims, s, s, 4, 255)
    save_im(img, 'temp.png')
    img = Image.open('temp.png')
    !rm temp.png
    return img

In [29]:
@interact
def inter_show(q_id=range(1052), show_num=range(1, 101)):
    return show(q_id, show_num)

interactive(children=(Dropdown(description='q_id', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, …

In [30]:
import json
import ntpath

In [31]:
ntpath.basename(img_list[0])[:-4]

'000000'

In [32]:
ans_ids = []
ans_list = gallery_np[ind[0]][1:]
for ans in ans_list:
    ans_ids.append(int(ntpath.basename(ans)[:-4]))

In [33]:
whole_ans = []
for idx, im in enumerate(img_list):
    query_id = int(ntpath.basename(im)[:-4])
    ans_ids = []
    ans_list = gallery_np[ind[idx]][1:]
    for ans in ans_list:
        ans_ids.append(int(ntpath.basename(ans)[:-4]))
    
    whole_ans.append({
        'query_id': query_id,
        'ans_ids': ans_ids
    })

In [34]:
json.dump(whole_ans, open('plain_reid_submit_rerank_all100.json', 'w'))